In [2]:
#import keras
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.models import load_model
from keras.applications import vgg16
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance
import random
import time
import urllib.request
import io
#import glob
from src.fetch_data_pipeline import extract_image_url, extract_df, download_images, load_RG_data, zip_lookup, gps_lookup
#import json
import requests

In [3]:
start = time.time()
feature_matrix = np.load('/Users/bil2ab/fetch_vectors_backup/fetch_feature_matrix.npy')
vector_list = pd.read_pickle('/Users/bil2ab/fetch_vectors_backup/fetch_vector_list.pkl', compression='gzip')
end = time.time()
print('Fetch Feature Matrix loaded.  Time: '+str(end-start))

Fetch Feature Matrix loaded.  Time: 25.39776587486267


In [4]:
start = time.time()
model = vgg16.VGG16(include_top = True, weights = 'imagenet')
model.layers.pop()
model.layers.pop()
model.outputs = [model.layers[-1].output]
end = time.time()
print('Neural Network initialized.  Time: '+str(end-start))

Neural Network initialized.  Time: 19.895319938659668


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
start = time.time()
yaml_model = model.to_yaml()
del model  # deletes the existing model
end = time.time()
print('Neural Network saved as YAML.  Time: '+str(end-start))

KeyError: 'predictions_ib-0'

In [ ]:
# returns a compiled model
# identical to the previous one
start = time.time()
model = load_model('test_model.h5')
end = time.time()
print('Neural Network loaded.  Time: '+str(end-start))

In [ ]:
filename_a = 'data/images/Dino.png'
#filename_a = 'data/test1.png'
#filename_a = 'data/test2.png'
#filename_a = 'data/test3.png'
#filename_a = 'data/mossy1.png'
#filename_a = 'data/Nibu.png'
#filename_a = 'data/dyno.png'
#filename_a = 'data/cool.png'

# Load image
original_a = load_img(filename_a, target_size=(224, 224))
print('PIL image size', original_a.size)
plt.imshow(original_a)
plt.show()
 
# Convert image to numpy array
# PIL - image (w, h, channel)  desxxNumpy - image (h, w, channel)
numpy_image_a = img_to_array(original_a)
plt.imshow(np.uint8(numpy_image_a))
plt.show()
print('numpy array size',numpy_image_a.shape)
 
# Convert the image into batch format
# expand_dims will add an extra dimension to the data at a particular axis
# We want the input matrix to the network to be of the form (batchsize, height, width, channels)
# Thus we add the extra dimension to the axis 0.
image_batch_a = np.expand_dims(numpy_image_a, axis=0)
print('image batch size', image_batch_a.shape)
plt.imshow(np.uint8(image_batch_a[0]))

In [ ]:
# prepare the image for the VGG model
processed_image_a = vgg16.preprocess_input(image_batch_a.copy())

# get the predicted probabilities for each class
predictions_a = model.predict(processed_image_a)

print(predictions_a)

In [ ]:
type(predictions_a)

In [ ]:
labels = []
for vector in vector_list:
    labels.append(vector[12:].split('.')[0]+'.jpg')

In [ ]:
start = time.time()

score_cos = distance.cdist(predictions_a, feature_matrix, 'cosine').tolist()
#score_manhattan = distance.cdist(predictions_a, feature_matrix, 'cityblock').tolist()
#score_ham = distance.cdist(predictions_a, feature_matrix, 'hamming').tolist()
#score_bc = distance.cdist(predictions_a, feature_matrix, 'braycurtis').tolist()
#score_euc = distance.cdist(predictions_a, feature_matrix, 'euclidean').tolist()
end = time.time()
print('Cosine Similarity with SciPy calculated.  Time: '+str(end-start))

In [ ]:
from numpy import linalg as LA

start = time.time()

score_cos_2 = (predictions_a * feature_matrix)/(LA.norm(predictions_a)*LA.norm(feature_matrix))
#distance.cdist(predictions_a, feature_matrix, 'cosine').tolist()

end = time.time()
print('Cosine Similarity calculated.  Time: '+str(end-start))

In [ ]:
score_cos

In [ ]:
#if len(labels) != len(score[0]):
#    print('Length mismatch!')

sorted_scores = sorted(list(zip(labels,score_cos[0])), key = lambda t: t[1])
    
for image, score in sorted_scores[0:10]:
    url = 'https://s3-us-west-2.amazonaws.com/hole-in-a-bucket/data/'+image
    im = Image.open(requests.get(url, stream=True).raw)
    plt.imshow(im)
    plt.show()

In [ ]:
#if len(labels) != len(score[0]):
#    print('Length mismatch!')

sorted_scores_cos_2 = sorted(list(zip(labels,score_cos_2[0])), key = lambda t: t[1])
    
for image, score in sorted_scores_cos_2[0:10]:
    url = 'https://s3-us-west-2.amazonaws.com/hole-in-a-bucket/data/'+image
    im = Image.open(requests.get(url, stream=True).raw)
    plt.imshow(im)
    plt.show()

In [ ]:
image = 

In [ ]:
# Top 10 - Random

for x in range(0,10):
    img_name = random.choice(vector_list)[12:].split('.')[0]+'.jpg'
    url ='https://s3-us-west-2.amazonaws.com/hole-in-a-bucket/data/'+img_name
    im = Image.open(requests.get(url, stream=True).raw)
    plt.imshow(im)
    plt.show()

In [ ]:
type(score)

In [ ]:
combined_df, combined_imgs = load_RG_data()

In [ ]:
test = 'durka.jpg'

In [ ]:
test.split('.')[0]

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
feature_matrix_test = np.load('data/feature_matrix/fetch_feature_matrix.npy')

In [ ]:
feature_matrix_test[0]

In [ ]:
feature_matrix_test.shape

In [ ]:
import numpy as np
idx = 35000

In [ ]:
if idx>0 and idx<140741:
    print(str(idx)+' vectors merged to feature matrix.')

In [ ]:
if idx%1000 == 0:
    print(str(idx)+' vectors merged to feature matrix.')

In [ ]:
a = np.linspace(0,5678,4096)
b = np.linspace(340,678,4096)
c = np.linspace(40,834789,4096)

In [ ]:
durka = np.vstack((feature_matrix_test[0],feature_matrix_test[2],feature_matrix_test[5],))

In [ ]:
durka

In [ ]:
durka.shape

In [ ]:
durka[0] = feature_matrix_test[7]

In [ ]:
def top_matches(labels,score):
    sorted_scores = sorted(list(zip(labels.tolist(),score)), key = lambda t: t[1])
    return sorted_scores[0:10]

In [ ]:
combined_df, combined_imgs = load_RG_data()

In [ ]:
# Extract image file name, store in Pandas Series and pickle for website
image_names = []
for image_url in combined_imgs.ImageUrl[0:140741]:
        image_names.append(image_url.split('/')[-1])

fetch_image_names = pd.Series(image_names)
fetch_image_names.to_pickle('data/fetch_img_urls.pkl', compression='gzip')



#combined_df.head()
#zipped_dogs = list(zip(dog_url.tolist(),results))
#sorted_zipped_dogs = sorted(zipped_dogs, key = lambda t: t[1])
#top_10 = sorted_zipped_dogs[0:11]
#top_10

In [ ]:
#Load NumPy data file of 4750 dogs and calculate average number of non-zero features in 1D arrays for all dogs

non_zero_features = []
feature_array_4750 = np.load('web/static/temp/data/doggie_features_4750.npy')

for dog in feature_array_4750:
    non_zero_features.append(len(np.where(dog>0)[1]))

plt.hist(non_zero_features)

In [ ]:
for scores in score_lists:
    for image,score in top_ten(dog_url, scores):
        plt.imshow(load_img('/Users/bil2ab/galvanize/RG5kimages/'+image.split('/')[-1]))
        plt.show()
        #print(1-score)
        #print('DURKA DURKA...next distance metric:')

In [ ]:

    start = time.time()
    feature_array_list = []
    #image_path_list formerly combined_df.ImageUrl[0:4750]
    for url in image_path_list[0:length]:
        image_path = '/Users/bil2ab/galvanize/capstone/images_to_vectorize/images/'+url.split('/')[-1]
        dog = load_img(image_path, target_size=(224, 224))
        numpy_image = img_to_array(dog)
        image_batch = np.expand_dims(numpy_image, axis=0)  
        processed_image = vgg16.preprocess_input(image_batch.copy())
        feature_array = model.predict(processed_image)
        feature_array_list.append(feature_array)
        #doggie = np.asarray(feature_array_list)
        #np.save('data/RG_50k_features', doggie)
    end = time.time()
    total_time = end-start
    print('Total Time: '+str(total_time))
    print('All dog features vectorized!')
    return feature_array_list

In [ ]:
url_vec = combined_imgs.ImageUrl.tolist()
holy_durka = vectorize_dog_images(url_vec, length=50000)
#len(url_50k)

In [ ]:
def durka():
    '''
    image_path_list = combined_df.ImageUrl.tolist()
    Take collection of dog images and vectorize each image to a 1D NumPy array. 
    INPUT: List, Pandas Series, some iterable of filepaths to dog images (strings)
    OUTPUT: Returns Numpy data file
    '''
    start = time.time()
    combined_df, combined_imgs = load_RG_data()
    #num_images = len(glob.glob1('/Users/bil2ab/galvanize/RG5kimages/','*.jpg'))
    image_path_list = combined_imgs.ImageUrl.tolist()
    feature_matrix = np.zeros((len(image_path_list),4096))
    
    for idx,url in enumerate(image_path_list):
        dog = load_img('/Users/bil2ab/galvanize/RG5kimages/'+url.split('/')[-1], target_size=(224, 224))
        image_batch = np.expand_dims(img_to_array(dog), axis=0)  
        processed_image = vgg16.preprocess_input(image_batch.copy())
        feature_matrix[idx] = model.predict(processed_image)
    
    #Save csv of image urls
    image_path_list.to_csv('/data/dog_urls_test.csv')
    
    #Save list of feature arrays as numpy data file
    #doggie = np.asarray(feature_array_list)
    np.save('/data/doggie_features_test', feature_matrix)
    
    end = time.time()
    print('Total Time: '+str(end-start))
    print('All dog features vectorized!')
    return feature_matrix

In [ ]:
feature_matrix

In [ ]:
%timeit np.dot(predictions_a, feature_matrix.T) / (np.linalg.norm(feature_matrix, axis=1) * np.linalg.norm(predictions_a))

In [ ]:
%timeit distance.cdist(predictions_a,feature_matrix,'cosine')[0]

In [ ]:
sorted_scores[0:10]

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

# Utility to search for layer index by name.
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'preds')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
plt.imshow(img[..., 0])

In [ ]:
#Load the VGG model
#vgg_model = vgg16.VGG16(weights='imagenet')

#Load the Inception_V3 model
#inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
#resnet_model = resnet50.ResNet50(weights='imagenet')
 
#Load the MobileNet model
#mobilenet_model = mobilenet.MobileNet(weights='imagenet')


In [ ]:
#Full vgg16
[<keras.engine.input_layer.InputLayer at 0x1aafbcf2b0>,
 <keras.layers.convolutional.Conv2D at 0x1ab00d4e80>,
 <keras.layers.convolutional.Conv2D at 0x1ab00d46d8>,
 <keras.layers.pooling.MaxPooling2D at 0x1ab0bd20f0>,
 <keras.layers.convolutional.Conv2D at 0x1aae8eedd8>,
 <keras.layers.convolutional.Conv2D at 0x1aaf356dd8>,
 <keras.layers.pooling.MaxPooling2D at 0x1aaf84ac88>,
 <keras.layers.convolutional.Conv2D at 0x1a7025d550>,
 <keras.layers.convolutional.Conv2D at 0x1aae42fa90>,
 <keras.layers.convolutional.Conv2D at 0x1aae8867b8>,
 <keras.layers.pooling.MaxPooling2D at 0x1aaddc9a58>,
 <keras.layers.convolutional.Conv2D at 0x1aac093940>,
 <keras.layers.convolutional.Conv2D at 0x1aaba4e2b0>,
 <keras.layers.convolutional.Conv2D at 0x1aaa1bccc0>,
 <keras.layers.pooling.MaxPooling2D at 0x1a6e7e6940>,
 <keras.layers.convolutional.Conv2D at 0x1a6dafbda0>,
 <keras.layers.convolutional.Conv2D at 0xb2f0ea0b8>,
 <keras.layers.convolutional.Conv2D at 0xb2f0ea2b0>,
 <keras.layers.pooling.MaxPooling2D at 0xb24c75c18>,
 <keras.layers.core.Flatten at 0xb18b03f28>,
 <keras.layers.core.Dense at 0xb2f333518>,
 <keras.layers.core.Dense at 0x1aaa750dd8>,
 <keras.layers.core.Dense at 0x1ab3fa36d8>]

#Original pop off
<keras.engine.input_layer.InputLayer at 0xb24363a90>,
 <keras.layers.convolutional.Conv2D at 0xb24350668>,
 <keras.layers.convolutional.Conv2D at 0xb17f81da0>,
 <keras.layers.pooling.MaxPooling2D at 0xb17f81c88>,
 <keras.layers.convolutional.Conv2D at 0xb18574080>,
 <keras.layers.convolutional.Conv2D at 0xb18574da0>,
 <keras.layers.pooling.MaxPooling2D at 0xb18592550>,
 <keras.layers.convolutional.Conv2D at 0xb185ab898>,
 <keras.layers.convolutional.Conv2D at 0xb185c7cf8>,
 <keras.layers.convolutional.Conv2D at 0xb185e23c8>,
 <keras.layers.pooling.MaxPooling2D at 0xb185fca20>,
 <keras.layers.convolutional.Conv2D at 0xb18619b70>,
 <keras.layers.convolutional.Conv2D at 0xb18633ef0>,
 <keras.layers.convolutional.Conv2D at 0xb1864e3c8>,
 <keras.layers.pooling.MaxPooling2D at 0xb18665780>,
 <keras.layers.convolutional.Conv2D at 0xb189c38d0>,
 <keras.layers.convolutional.Conv2D at 0xb189e0c50>,
 <keras.layers.convolutional.Conv2D at 0xb189e0f60>,
 <keras.layers.pooling.MaxPooling2D at 0xb18a0d6d8>,
 <keras.layers.core.Flatten at 0xb18a2d630>,
 <keras.layers.core.Dense at 0xb18a2dd30>

In [ ]:
#Function to extract exif data from smartphone image and view in nice format

from PIL.ExifTags import TAGS

#def extract_image_data(file):
filename =''
im = PIL.Image.open(filename)
exifdict = im._getexif()
#print(exifdict)

if len(exifdict):
    for k in exifdict.keys():
        if k in TAGS.keys():
            print(TAGS[k], exifdict[k])
        else:
            print(k, exifdict[k])